## Reqs

In [ ]:
!git clone https://github.com/NeuralClassifier/ReusabilityRate_ML
!git checkout corr_analysis
%cd /content/ReusabilityRate_ML/DataSets

Cloning into 'ReusabilityRate_ML'...
remote: Enumerating objects: 150, done.
remote: Counting objects: 100% (150/150), done.
remote: Compressing objects: 100% (142/142), done.
remote: Total 150 (delta 63), reused 15 (delta 6), pack-reused 0
Receiving objects: 100% (150/150), 9.04 MiB | 10.91 MiB/s, done.
Resolving deltas: 100% (63/63), done.
fatal: not a git repository (or any of the parent directories): .git
/content/ReusabilityRate_ML/DataSets


In [ ]:
import pandas as pd
import numpy as np


from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold 
from sklearn.metrics import mean_absolute_error 
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import statistics
import tensorflow as tf

import warnings
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

### Read class and package metrics(x:inputs) and RR(y:label)

In [ ]:
cls_df = pd.read_csv('cls_normalized.csv')
pkg_df = pd.read_csv('pkg_normalized.csv')
cls_df.drop(['Unnamed: 0'], axis=1, inplace=True)
pkg_df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
pkg_df

,Project,LongName,Name,AD,CD,TCD,CLOC,TCLOC,PDA,TPDA,LLOC,LOC,TLOC,NG,TLLOC,TNA,TNG,TNM,TNOS,TNPM,NCL,ReuseRate,ReuseRate_lbl
0,ch.qos.logback/logback-classic,<root_package>,<root_package>,-1.000000,-1.000000,-0.255446,-1.000000,-0.959769,-1.000000,-0.961194,-1.000000,-1.000000,-0.956749,-1.000000,-0.959630,-0.908012,-0.952302,-0.956669,-0.941933,-0.962812,-1.000000,0.0,0
1,ch.qos.logback/logback-classic,ch,ch,-1.000000,-1.000000,-0.253473,-1.000000,-0.959769,-1.000000,-0.961194,-1.000000,-1.000000,-0.956864,-1.000000,-0.959792,-0.908012,-0.952302,-0.956669,-0.941933,-0.962812,-1.000000,0.0,0
2,ch.qos.logback/logback-classic,unnamed package,unnamed package,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-0.999661,-0.999712,-0.999885,-1.000000,-0.999838,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.0,0
3,ch.qos.logback/logback-classic,ch.qos,qos,-1.000000,-1.000000,-0.253473,-1.000000,-0.959769,-1.000000,-0.961194,-1.000000,-1.000000,-0.956864,-1.000000,-0.959792,-0.908012,-0.952302,-0.956669,-0.941933,-0.962812,-1.000000,0.0,0
4,ch.qos.logback/logback-classic,ch.qos.logback,logback,-1.000000,-1.000000,-0.253473,-1.000000,-0.959769,-1.000000,-0.961194,-1.000000,-1.000000,-0.956864,-1.000000,-0.959792,-0.908012,-0.952302,-0.956669,-0.941933,-0.962812,-1.000000,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2188,xerces/xercesImpl,org.apache.xerces.impl.xs.identity,identity,0.968254,0.033413,0.083579,-0.977705,-0.993081,-0.931073,-0.989014,-0.992390,-0.987862,-0.995148,-0.940460,-0.996360,-0.991098,-0.995164,-0.996647,-0.993460,-0.996841,-0.977604,0.0,0
2189,xerces/xercesImpl,org.apache.xerces.impl.xs.models,models,0.402986,-0.011642,0.036337,-0.964447,-0.988967,-0.947749,-0.991672,-0.986761,-0.979412,-0.991769,-0.953992,-0.993669,-0.988131,-0.996263,-0.995377,-0.987849,-0.996484,-0.977604,0.0,0
2190,xerces/xercesImpl,org.apache.xerces.impl.xs.opti,opti,0.081272,-0.000149,0.048387,-0.940306,-0.981476,-0.829906,-0.972889,-0.978261,-0.965010,-0.986012,-0.745602,-0.989603,-0.978087,-0.979338,-0.984151,-0.985873,-0.983313,-0.966405,3.0,2
2191,xerces/xercesImpl,org.apache.xerces.impl.xs.traversers,traversers,0.400000,-0.532498,-0.509803,-0.920482,-0.975324,-0.984436,-0.997519,-0.907371,-0.904436,-0.961795,-0.907984,-0.955701,-0.924218,-0.992527,-0.985268,-0.919689,-0.996603,-0.935050,0.0,0


## Class DNN

### Load data

In [ ]:
data = cls_df
df = data.iloc[:,3:-2]
data_columns = df.columns
X_np = np.array(df)
X = X_np

Y = data.iloc[:,-1]

In [ ]:
data

,Project,LongName,Name,LCOM5,NL,NLE,WMC,CBO,CBOI,NII,NOI,RFC,AD,CD,TCD,CLOC,TCLOC,DLOC,PDA,DIT,LLOC,LOC,TLOC,NG,TLLOC,TNA,TNG,TNM,TNOS,TNPM,ReuseRate,ReuseRate_lbl
0,ch.qos.logback/logback-classic,ch.qos.logback.classic.AsyncAppender,AsyncAppender,-0.976331,-0.916667,-0.8,-0.990539,-0.982979,-1.000000,-1.000000,-0.984127,-0.979975,-0.333334,-0.110417,-0.110417,-0.995522,-0.995522,-0.995389,-0.993651,-1.000000,-0.991793,-0.993978,-0.993978,-1.000000,-0.995344,-0.997199,-1.0000,-0.995408,-0.996554,-0.997253,5.307617,4
1,ch.qos.logback/logback-classic,ch.qos.logback.classic.BasicConfigurator,BasicConfigurator,-0.988166,-1.000000,-1.0,-0.996216,-0.957447,-0.996370,-0.998538,-0.988095,-0.987484,-0.333334,-0.322223,-0.322223,-0.996802,-0.996802,-0.997695,-0.993651,-0.777778,-0.990828,-0.993548,-0.993548,-1.000000,-0.994796,-1.000000,-1.0000,-0.997704,-0.993109,-0.997253,4.000000,3
2,ch.qos.logback/logback-classic,ch.qos.logback.classic.ClassicConstants,ClassicConstants,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-0.974592,-1.000000,-1.000000,-1.000000,-1.000000,-0.346428,-0.346428,-0.997121,-0.997121,-0.997036,-1.000000,-1.000000,-0.991311,-0.992687,-0.992687,-1.000000,-0.995070,-0.952381,-1.0000,-1.000000,-1.000000,-1.000000,5.307617,4
3,ch.qos.logback/logback-classic,ch.qos.logback.classic.Level,Level,-0.988166,-0.916667,-0.8,-0.912961,-1.000000,-0.883848,-0.970760,-1.000000,-0.967459,1.000000,-0.242484,-0.242484,-0.969615,-0.969615,-0.969043,-0.923810,-1.000000,-0.923244,-0.941284,-0.941284,-1.000000,-0.956450,-0.932773,-1.0000,-0.985075,-0.966527,-0.984890,5.307617,4
4,ch.qos.logback/logback-classic,ch.qos.logback.classic.Logger,Logger,-0.988166,-0.750000,-0.4,-0.708609,-0.957447,-0.916515,-0.969298,-0.956349,-0.732165,-0.851852,-0.581752,-0.581752,-0.958420,-0.958420,-0.960481,-0.961905,-1.000000,-0.758146,-0.838047,-0.838047,-0.929293,-0.862777,-0.971989,-0.9375,-0.889782,-0.877923,-0.890110,5.307617,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24789,xerces/xercesImpl,org.apache.xml.serialize.SecuritySupport$4,SecuritySupport$4,-0.988166,-1.000000,-1.0,-0.998108,-1.000000,-1.000000,-1.000000,-1.000000,-0.997497,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-0.998069,-0.999140,-0.999140,-1.000000,-0.998904,-1.000000,-1.0000,-0.998852,-0.999508,-0.998626,0.000000,0
24790,xerces/xercesImpl,org.apache.xml.serialize.SecuritySupport$5,SecuritySupport$5,-0.988166,-1.000000,-1.0,-0.998108,-1.000000,-1.000000,-1.000000,-1.000000,-0.997497,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-0.998069,-0.999140,-0.999140,-1.000000,-0.998904,-1.000000,-1.0000,-0.998852,-0.999508,-0.998626,0.000000,0
24791,xerces/xercesImpl,org.apache.xml.serialize.SecuritySupport$6,SecuritySupport$6,-0.988166,-0.916667,-0.8,-0.996216,-1.000000,-1.000000,-1.000000,-1.000000,-0.997497,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-0.995173,-0.997849,-0.997849,-1.000000,-0.997261,-1.000000,-1.0000,-0.998852,-0.997539,-0.998626,0.000000,0
24792,xerces/xercesImpl,org.apache.xml.serialize.SecuritySupport$7,SecuritySupport$7,-0.988166,-1.000000,-1.0,-0.996216,-1.000000,-1.000000,-1.000000,-1.000000,-0.997497,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-0.998069,-0.999140,-0.999140,-1.000000,-0.998904,-1.000000,-1.0000,-0.998852,-0.999508,-0.998626,0.000000,0


In [ ]:
data_columns

Index(['LCOM5', 'NL', 'NLE', 'WMC', 'CBO', 'CBOI', 'NII', 'NOI', 'RFC', 'AD',
       'CD', 'TCD', 'CLOC', 'TCLOC', 'DLOC', 'PDA', 'DIT', 'LLOC', 'LOC',
       'TLOC', 'NG', 'TLLOC', 'TNA', 'TNG', 'TNM', 'TNOS', 'TNPM'],
      dtype='object')

In [ ]:
cls_correletad_idx = [5, 12, 13, 14, 15]

### Training data preparation 

In [ ]:
splitter = int(len(X) * .8)
X_train = X[:splitter].astype('float32')
Y_train = Y[:splitter].astype('int32')
X_val = X[splitter:].astype('float32')
Y_val = Y[splitter:].astype('int32')

### Train model

In [ ]:
##model

def my_model_fully_connected(loss_fn, opt, ishape = 27, lr = 1e-2):
    model_input = tf.keras.layers.Input(shape = ishape)
    l2 = tf.keras.layers.Dense(50, activation = 'relu')(model_input)
    l2_drop = tf.keras.layers.Dropout(0.02)(l2)
    l3 = tf.keras.layers.Dense(20, activation = 'relu')(l2_drop)
    l4 =  tf.keras.layers.Dense(20, activation = 'relu')(l3)
    l5 =  tf.keras.layers.Dense(10, activation = 'relu')(l4)
    #l6 =  tf.keras.layers.Dense(10, activation = 'relu')(l5)
    out= tf.keras.layers.Dense(1, activation = 'softmax')(l5)
    model = tf.keras.models.Model(inputs = model_input, outputs = out)
    model.compile(optimizer=opt, loss=loss_fn, metrics = ['accuracy'])
    return model

lr = 0.001
opt = 'adam'
loss_fn = 'mse'
model = my_model_fully_connected(loss_fn, opt, ishape=18, lr= lr)
print(model.summary())


Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 18)]              0         
                                                                 
 dense_17 (Dense)            (None, 50)                950       
                                                                 
 dropout_3 (Dropout)         (None, 50)                0         
                                                                 
 dense_18 (Dense)            (None, 20)                1020      
                                                                 
 dense_19 (Dense)            (None, 20)                420       
                                                                 
 dense_20 (Dense)            (None, 10)                210       
                                                                 
 dense_21 (Dense)            (None, 1)                 11  

In [ ]:
# Call backs 
model_name_fcn = "First_draft.h5"
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 20)

monitor = tf.keras.callbacks.ModelCheckpoint(model_name_fcn, monitor='val_loss',\
                                             verbose=0, save_best_only=True,\
                                             save_weights_only=True,\
                                             mode='min')
# Learning rate schedule
def scheduler(epoch, lr):
    if epoch%10 == 0:
        lr = lr/2
    return lr

lr_schedule = tf.keras.callbacks.LearningRateScheduler(scheduler, verbose = 0)

In [ ]:
# Train
model.fit(X_train, Y_train, epochs = 100, batch_size=200, \
          verbose = 1, callbacks= [early_stop, monitor, lr_schedule], validation_data= (X_val, Y_val))

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

clf = MLPClassifier(hidden_layer_sizes=(200, 100, 50, 20, 10), random_state=1, max_iter=300, solver='adam').fit(X_train, Y_train)

clf.score(X_val, Y_val)

0.7469247832224238

In [ ]:
X_train

In [ ]:

for i in range(len(X_train[0])):
  print('feature: {0} \n'.format(i))
  X_temp_train = np.zeros((len(X_train), len(X_train[0])))
  X_temp_val =  np.zeros((len(X_val), len(X_val[0])))

  X_temp_train[:, i] = X_train [:,i]
  X_temp_val[:, i] = X_val [:,i]
  clf = MLPClassifier(hidden_layer_sizes=(200, 100, 50, 20, 10), random_state=1, max_iter=600, solver='adam').fit(X_temp_train, Y_train)
  print(clf.score(X_temp_val, Y_val))
  

In [ ]:
X_temp_train = np.zeros((len(X_train), len(X_train[0])))
X_temp_train[:, 0] = X_train [:,0]
X_temp_train

### Regression

In [ ]:
data = cls_df
df = data.iloc[:,3:-2]
data_columns = df.columns
X_np = np.array(df)
X = X_np

Y = data.iloc[:,-2]


splitter = int(len(X) * .8)
X_train = X[:splitter].astype('float32')
Y_train = Y[:splitter].astype('int32')
X_val = X[splitter:].astype('float32')
Y_val = Y[splitter:].astype('int32')

corr_data = True

if corr_data:
  X_temp_train = np.zeros((len(X_train), len(cls_correletad_idx)))
  X_temp_val =  np.zeros((len(X_val), len(cls_correletad_idx)))

  counter = 0
  for i in cls_correletad_idx:  
    X_temp_train[:,counter] = X_train [:,i]
    X_temp_val[:, counter] = X_val [:,i]
    counter += 1
  X_train = X_temp_train
  X_val = X_temp_val

In [ ]:
X_train

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

clf = MLPRegressor(hidden_layer_sizes=(200, 100, 50, 20, 10), random_state=1, max_iter=600, solver='adam').fit(X_train, Y_train)

Y_pred = clf.predict(X_val)
rms = mean_squared_error(Y_val, Y_pred, squared=False)
rms

1.618241740627832

## Package DNN

### Load data

In [ ]:
data = pkg_df
df = data.iloc[:,3:-2]
data_columns = df.columns
X_np = np.array(df)
X = X_np

Y = data.iloc[:,-1]
data_columns

In [ ]:
cls_correletad_idx = [1, 2, 3, 5, 17]

In [ ]:
splitter = int(len(X) * .8)
X_train = X[:splitter]
Y_train = Y[:splitter]
X_val = X[splitter:]
Y_val = Y[splitter:]

corr_data = False

if corr_data:
  X_temp_train = np.zeros((len(X_train), len(cls_correletad_idx)))
  X_temp_val =  np.zeros((len(X_val), len(cls_correletad_idx)))

  counter = 0
  for i in cls_correletad_idx:  
    X_temp_train[:,counter] = X_train [:,i]
    X_temp_val[:, counter] = X_val [:,i]
    counter += 1
  X_train = X_temp_train
  X_val = X_temp_val

## Train model

### REG

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

clf = MLPRegressor(hidden_layer_sizes=(100, 50, 20, 10), random_state=1, max_iter=600, solver='adam').fit(X_train, Y_train)

Y_pred = clf.predict(X_val)
rms = mean_squared_error(Y_val, Y_pred, squared=False)
rms

### Classification


In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

clf = MLPClassifier(hidden_layer_sizes=(100, 50, 20, 10), random_state=1, max_iter=600, solver='adam').fit(X_train, Y_train)
clf.score(X_val, Y_val)